In [ ]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import CRUD module
from AnimalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################

# MongoDB authentication
username = "root"
password = "x1vXfXIAAm"

# Connect to MongoDB using CRUD module
db = AnimalShelter(username, password)

# Load all records initially
df = pd.DataFrame.from_records(db.read({}))

# Drop '_id' column from MongoDB
#df.drop(columns=['_id'], inplace=True)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('Simple')

# Load Grazioso Salvare logo
image_filename = 'logo.png'  # Replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()

# Dashboard layout
app.layout = html.Div([
    html.Center(html.Img(src=f'data:image/png;base64,{encoded_image}', style={'width': '200px'})),
    html.Center(html.B(html.H1('Nhat-CS-340 Dashboard'))),
    html.Hr(),
    
    html.Div([
        html.Label("Select Rescue Type:"),
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'All', 'value': 'all'},
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain/Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
                {'label': 'Disaster/Individual Tracking', 'value': 'Disaster or Individual Tracking'}
            ],
            value='all',
            inline=True
        )
    ]),

    html.Hr(),

    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        page_size=10,
        row_selectable="single",
        style_table={'overflowX': 'auto'}
    ),

    html.Br(),
    html.Hr(),

    # Graphs and Map Layout
    html.Div(
        className='row',
        style={'display': 'flex'},
        children=[
            html.Div(id='graph-id', className='col s12 m6'),
            html.Div(id='map-id', className='col s12 m6')
        ]
    )
])

#############################################
# Interaction Between Components / Controller
#############################################

# Update data table based on selected rescue type
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    if filter_type == 'all':
        filtered_df = pd.DataFrame.from_records(db.read({}))
    else:
        filtered_df = pd.DataFrame.from_records(db.read({"breed": filter_type}))

    filtered_df.drop(columns=['_id'], inplace=True)

    return filtered_df.to_dict('records')

# Display breed distribution in pie chart
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    if viewData is None:
        return "No data available"
    
    dff = pd.DataFrame.from_dict(viewData)
    fig = px.pie(dff, names='breed', title='Rescue Dog Breeds Distribution')
    
    return [dcc.Graph(figure=fig)]

# Highlight selected column in data table
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Update map based on selected row
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):  
    if viewData is None or index is None:
        return "No location data available"
    
    dff = pd.DataFrame.from_dict(viewData)
    
    # Select the first row if no row is selected
    row = index[0] if index else 0
    
    return [
        dl.Map(
            style={'width': '1000px', 'height': '500px'},
            center=[30.75, -97.48],
            zoom=10,
            children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(
                    position=[dff.iloc[row, 13], dff.iloc[row, 14]],
                    children=[
                        dl.Tooltip(dff.iloc[row, 4]),
                        dl.Popup([
                            html.H1("Animal Name"),
                            html.P(dff.iloc[row, 9])
                        ])
                    ]
                )
            ]
        )
    ]

# Run Dash app
app.run_server(debug=True)
